In [75]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [76]:
# open the .txt file which have names of training videos
f = open("video-output/videolist.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train.head()

,video_name
0,pose-p1-1.mp4
1,pose-p1-2.mp4
2,pose-p1-3.mp4
3,pose-p1-4.mp4
4,pose-p1-5.mp4


In [77]:
# open the .txt file which have names of test videos
f = open("video-output/testlist.txt", "r")
temp = f.read()
videos = temp.split('\n')


# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test.head()

,video_name
0,pose-testing-1.mp4
1,pose-testing-2.mp4
2,pose-testing-3.mp4


In [78]:
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('.')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('.')[0])
    
test['tag'] = test_video_tag

In [81]:
# storing the frames from training videos
for i in tqdm(range(train.shape[0])):    
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('video-output/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate/4) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile+f"_frame{count}.jpg"
            count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 10/10 [00:47<00:00,  4.78s/it]


In [85]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i])
    # creating the class of image
    train_class.append(images[i].split('_')[1].split("\\")[1].split('-')[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

100%|██████████| 9101/9101 [00:00<00:00, 758486.71it/s]


In [ ]:
# After preprocessing the Data we move on to training our model!

In [83]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [86]:
train = pd.read_csv('train_new.csv')
train.head()

,image,class
0,train_1\pose-p1-1.mp4_frame0.jpg,p1
1,train_1\pose-p1-1.mp4_frame1.jpg,p1
2,train_1\pose-p1-1.mp4_frame10.jpg,p1
3,train_1\pose-p1-1.mp4_frame100.jpg,p1
4,train_1\pose-p1-1.mp4_frame101.jpg,p1


In [87]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img(train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 9101/9101 [02:06<00:00, 71.74it/s]


(9101, 224, 224, 3)

In [88]:
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

In [89]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [90]:
# creating the base model of pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

In [91]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

(7280, 7, 7, 512)

In [92]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(1821, 7, 7, 512)

In [94]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(7280,7*7*512)
X_test = X_test.reshape(1821,7*7*512)

In [95]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [96]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [104]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [105]:
# compiling the model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [106]:
# training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)


Epoch 1/10
57/57 [==============================] - 7s 113ms/step - loss: 7.3698e-04 - accuracy: 0.9997 - val_loss: 0.0019 - val_accuracy: 0.9989
Epoch 2/10
57/57 [==============================] - 6s 106ms/step - loss: 9.3527e-04 - accuracy: 0.9995 - val_loss: 0.0016 - val_accuracy: 0.9989
Epoch 3/10
57/57 [==============================] - 6s 106ms/step - loss: 0.0083 - accuracy: 0.9988 - val_loss: 0.0017 - val_accuracy: 0.9989
Epoch 4/10
57/57 [==============================] - 6s 107ms/step - loss: 0.0209 - accuracy: 0.9978 - val_loss: 0.0083 - val_accuracy: 0.9978
Epoch 5/10
57/57 [==============================] - 6s 105ms/step - loss: 0.0021 - accuracy: 0.9993 - val_loss: 0.0055 - val_accuracy: 0.9984
Epoch 6/10
57/57 [==============================] - 6s 105ms/step - loss: 0.0021 - accuracy: 0.9998 - val_loss: 0.0014 - val_accuracy: 0.9989
Epoch 7/10
57/57 [==============================] - 6s 106ms/step - loss: 0.0016 - accuracy: 0.9989 - val_loss: 0.0015 - val_accuracy: 0.998

In [107]:
model.save('final_model.model')

INFO:tensorflow:Assets written to: final_model.model\assets


In [110]:
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size = 128)
print(f"Loss: {val_loss}")
print(f"Accuracy: {val_acc * 100}")

15/15 [==============================] - 1s 67ms/step - loss: 0.0011 - accuracy: 0.9989
Loss: 0.0011362413642928004
Accuracy: 99.89017248153687
